# <center> Data Wrangling OpenStreetMap with SQL</center>

<center> By: Kyle Hansen </center>



## <center>Map Area</center>
#### <center>Bothell, Washington, USA</center>

<img src="https://i.imgur.com/EnMjSce.jpeg" alt="Map" style="width: 550px;"/>
<center><a href="https://www.openstreetmap.org/relation/237656">https://www.openstreetmap.org/relation/237656</a></center>


<b>Project Summary:</b>
For this project, using data wrangling techniques, i chose Bothell, WA map area from OpenStreetMap.org.  The reason for this choice is because i grew up in Bothell.  The cleaned, fixed data starts in XML and is converted into CSV format, and is then imported into a SQL database.


### Tag Count

When Parsing through the Bothell, WA dataset and couting the unique element types by using the tags function, the following are the number of unique tags:

In [16]:
osm_file = "bothell.osm"
import tags
import tag_type
import audit
import pandas as pd

In [2]:
tags.count_tags(osm_file)

defaultdict(int,
            {'bounds': 1,
             'member': 34048,
             'meta': 1,
             'nd': 700084,
             'node': 607354,
             'note': 1,
             'osm': 1,
             'relation': 994,
             'tag': 363047,
             'way': 79027})

### Tag Patterns

There are three regular expression, lower is for tags that contain only lowercase letters and are valid. lower_colon is for other valid tags with a colon in the value. problem is for tags with problematic characters. The results are as below:

In [3]:
tag_type.process_map(osm_file)

{'lower': 165323, 'lower_colon': 194710, 'other': 3014, 'problem': 0}

## <center>Problems Seen in the Data</center>

### Street Types:

Because the data is inputed by users, there is no standard format for names.  With a list of expected street types used, we can use the audit function to see what street addresses have unexpected types.  After running the function, the result shows the majority of the issues involve abbreviations such as St for Street.

<b>Example from output:</b>

             'Pl': {'N 202nd Pl'},
             'SE': {'17th Ave SE',
              '208th St SE',
              '27th Ave SE',
              '37th DR SE',
              '39th Dr. SE'},
             'SW': {'196th St SW', '196th Street SW', '236th St SW'},
             'South': {'Meridian Avenue South', 'North Creek Parkway South'},
             'Southeast': {'10th Avenue Southeast',

To fix the abbreviations in the street names, to update with full names, a mapping dictionary can be used:

In [4]:
mapping = { "Ave": "Avenue",
            "Ave.": "Avenue",
            "avenue": "Avenue",
            "ave": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
            "Blvd,": "Boulevard",
            "Boulavard": "Boulevard",
            "Boulvard": "Boulevard",
            "Ct": "Court",
            "Dr": "Drive",
            "Dr.": "Drive",
            "E": "East",
            "Hwy": "Highway",
            "Ln": "Lane",
            "Ln.": "Lane",
            "N": "North",
            "Pl": "Place",
            "Plz": "Plaza",
            "Rd": "Road",
            "Rd.": "Road",
            "S:": "South",
            "St": "Street",
            "St.": "Street",
            "st": "Street",
            "street": "Street",
            "square": "Square",
            "parkway": "Parkway",
            "W": "West",
            "NW": "Northwest",
            "NE": "Northeast",
            "SW": "Southwest",
            "sw": "Southwest",
            "SE": "Southeast",
            "state": "State",
            "99": "Highway 99",
            "WA-99":"Highway 99"
            }


In [5]:
audit.fix_street(osm_file)

('NE 145th St.', '=>', 'NE 145th Street')
('N 202nd Pl', '=>', 'N 202nd Place')
('25th Ave NE', '=>', '25th Ave Northeast')
('Ballinger Way NE', '=>', 'Ballinger Way Northeast')
('Juanita-Woodinville Way NE', '=>', 'Juanita-Woodinville Way Northeast')
('19115 112th Ave NE', '=>', '19115 112th Ave Northeast')
('5th Ave NE', '=>', '5th Ave Northeast')
('94th Pl NE', '=>', '94th Pl Northeast')
('104th Ave NE', '=>', '104th Ave Northeast')
('141st Pl NE', '=>', '141st Pl Northeast')
('68th Ave NE', '=>', '68th Ave Northeast')
('138th Way NE', '=>', '138th Way Northeast')
('Bothell Way NE', '=>', 'Bothell Way Northeast')
('120th Avenue NE', '=>', '120th Avenue Northeast')
('127th Ave NE', '=>', '127th Ave Northeast')
('Bothell-Everett Hwy', '=>', 'Bothell-Everett Highway')
('196th St SW', '=>', '196th St Southwest')
('236th St SW', '=>', '236th St Southwest')
('196th Street SW', '=>', '196th Street Southwest')
('Martin Way E', '=>', 'Martin Way East')
('NE 185th St', '=>', 'NE 185th Street'

### Postal Codes

We also can look at the postal codes in the data.  The zip codes themselves look pretty clean, other than 6 zip codes are in a longer than 5 digit format. After cleaning the extra digits off the long zip codes, there are a total of 16 extra zip codes that are not in the Bothell zip code database from <a href="http://www.unitedstateszipcodes.org/zip-code-database/">http://www.unitedstateszipcodes.org/zip-code-database/</a>.

In [4]:
pc = audit.audit_postcode(osm_file)
print 'Total: ' + str(len(pc)) + ' zip codes outside of Bothell.'
post_codes = pd.DataFrame(data=pc, columns=['Zip Code'])
post_codes

There are 6 long post codes.
Total: 16 zip codes outside of Bothell.


,Zip Code
0,98034
1,98936
2,98296
3,98038
4,98028
5,98026
6,98133
7,98125
8,98503
9,98072


## <center>Data Overview </center>

##### Basic Statistics of the Data:

<b>Bothell.osm</b> 141,444 KB<br>
<b>Bothell.db</b> 75,838 KB<br>
<b>nodes.csv</b> 54,103 KB<br>
<b>nodes_tags.csv</b> 2,134 KB<br>
<b>ways.csv</b> 5,083 KB<br>
<b>ways_nodes.csv</b> 17,096 KB<br>
<b>ways_tags.csv</b> 10,487 KB<br>



In [6]:
import query
import pandas as pd
import pprint

### Number of Nodes

In [ ]:
query = ''' SELECT COUNT(*) FROM nodes '''

In [7]:
print 'Total Nodes: ' + str(query.num_nodes())

Total Nodes: 607354


### Number of Ways

In [ ]:
query = ''' SELECT COUNT(*) FROM ways '''

In [8]:
print 'Total Ways: ' + str(query.num_ways())

Total Ways: 79027


### Most Common Ways

In [ ]:
query = '''SELECT key, count(*)
            FROM ways_tags 
            GROUP BY 1 
            ORDER BY count(*) DESC 
            LIMIT 5
         '''

In [9]:
print 'Most Common Ways: '
d = query.common_way_tag()
df = pd.DataFrame(data=d, columns=['Way', 'Count'])
df

Most Common Ways: 


,Way,Count
0,building,47771
1,housenumber,30404
2,street,30068
3,city,29952
4,postcode,29291


### Number of Unique Users

In [ ]:
query = ''' SELECT COUNT(distinct(uid)) 
            FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways)'''

In [10]:
print 'Number of Unique Users: ' + str(query.num_unique_users())

Number of Unique Users: 1276


### Number of Unique Bars

In [ ]:
query = '''  SELECT COUNT(distinct id) FROM nodes_tags WHERE value="bar"  '''

In [11]:
print 'Number of Unique Bars: ' + str(query.num_unique_bars())

Number of Unique Bars: 8


### Number of Unique Schools

In [ ]:
query = '''  SELECT COUNT(*) FROM nodes_tags WHERE value="school"  '''

In [12]:
print 'Number of Unique Schools: ' + str(query.num_unique_schools())

Number of Unique Schools: 37


### Top Contributers

In [ ]:
query = '''SELECT user, COUNT(*) as num 
           FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) user 
           GROUP BY user 
           ORDER BY num DESC 
           LIMIT 5
        '''

In [13]:
print 'Top Contributers: '
d = query.top_contributer()
df = pd.DataFrame(data=d, columns=['User', 'Count'])
df

Top Contributers: 


,User,Count
0,patricknoll_import,238998
1,Glassman_Import,50865
2,SeattleImport,44542
3,seattlefyi_import,43378
4,Natfoot,30387


### Most Popular Religions

In [ ]:
query = '''SELECT nodes_tags.value, COUNT(*) as num 
           FROM nodes_tags 
           JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="place_of_worship") i 
           ON nodes_tags.id=i.id 
           WHERE nodes_tags.key="religion" 
           GROUP BY nodes_tags.value 
           ORDER BY num DESC
           LIMIT 2
        '''

In [14]:
print 'Top Religions: '
d = query.most_popular_religion()
df = pd.DataFrame(data=d, columns=['Relgion', 'Count'])
df

Top Religions: 


,Relgion,Count
0,christian,14
1,muslim,1


### Most Popular Amenity

In [ ]:
query = '''SELECT value, COUNT(distinct id) as num 
           FROM nodes_tags 
           WHERE key="amenity" 
           GROUP BY value 
           ORDER BY num DESC 
           LIMIT 10
        '''

In [15]:
print 'Top Amenties: '
d = query.most_popular_amenity()
df = pd.DataFrame(data=d, columns=['Amenity', 'Count'])
df

Top Amenties: 


,Amenity,Count
0,restaurant,149
1,bench,95
2,fast_food,80
3,cafe,60
4,waste_basket,38
5,school,36
6,bank,32
7,bicycle_parking,30
8,clinic,25
9,toilets,23


## <center>Additional Ideas</center>

<ol>
   <b><li>Create a standard format for street addresses by autocorrecting abbreviations or allowing the user to choose from a drop down list of options when submitting address</li></b>
    The issue for creating a stricter format for inputing contributions would be that it would cost time and money for developement of an updated website interface for doing so.
   <b><li>Allow data from outside sources to be imported to expand the information on amentities, attractions, etc.</li></b>
           One issue that would occur from allowing outside sources would be matching up the data and making sure the format is inline with the current data
     
</ol>